In [1]:
from datasketch.hnsw import HNSW
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
name = 'mnist-784'
runs = 1
metric = 'euclidean'
queries = 1000
nameFull = name + '-' + metric + '-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (60000, 784)
testDataset :  (10000, 784)


***Create datascetch***

In [3]:
def createIndex(indexMethod, datasetImages):
    time_start = perf_counter()
    index = indexMethod(distance_func=lambda x, y: np.linalg.norm(x - y))
    for i, d in enumerate(datasetImages):
        index.insert(i, d)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    return (index, totalTime)
(indexedStruct, time) = createIndex(HNSW, datasetTrainImages)

Took 876.377 seconds


In [5]:
(minBuildTime, maxBuildTime, indexedStruct) = createIndexNumerous(createIndex, HNSW, datasetTrainImages, runs)
print('minBuildTime : ', minBuildTime)
print('maxBuildTime : ', maxBuildTime)

In [6]:
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        result = indexedStruct.query(xq, k=100)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        index = []
        distance = []
        for j in range(len(result)):
            index.append(result[j][0])
            distance.append(result[j][1])
        distances.append(distance)
        indexes.append(index)
    return np.round(totalTime, 3)

Took 18.067 seconds


In [7]:
(minSearchTime, maxSearchTime, indexes, distances) = measureTimeNumerous(measureTime, runs, queries, datasetTestImages)
print('minSearchTime : ', minSearchTime)
print('maxSearchTime : ', maxSearchTime)

In [8]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [9]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [10]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [11]:
amount = 10
compareElems(amount, indexes, distances, trueIndexes, trueDistances)

53843 || 53843
676.584 || 676.584
38620 || 38620
793.9868 || 793.9868
16186 || 16186
862.6766 || 862.6766
27059 || 27059
864.5039 || 864.5039
47003 || 47003
894.7 || 894.7
14563 || 14563
909.7043 || 909.7043
44566 || 44566
917.6323 || 917.6323
15260 || 15260
921.6241 || 921.6241
40368 || 40368
922.147 || 922.147
36395 || 36395
943.4972 || 943.4972


In [12]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.9622
0.9999
1.0000
